# This notebook can be executed after getRegionalBox.ipynb

In [2]:
using Pkg

cd(@__DIR__)
Pkg.activate("../")
include("../src/commonBatchs.jl")
using .commonBatchs

  Activating project at `~/Documents/Github/flexOPT`


In [ ]:
# what you need as a model:

_, allGridsInCartesianMar, _ = lazyProduceOrLoad("MarmousiCoordInfo"); # for a big model, I do not recommend to store these data

seismicModelMarmousi=lazyProduceOrLoad("seismicModelMarmousi");

This is the laziest ProduceOrLoad
Loading from ./tmp/MarmousiCoordInfo.jld2


┌ Warning: type Main.GeoPoints.GeoPoint does not exist in workspace; reconstructing
└ @ JLD2 /Users/nobuaki/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588
┌ Warning: type Main.GeoPoints.localCoord2D does not exist in workspace; reconstructing
└ @ JLD2 /Users/nobuaki/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588
┌ Warning: type Main.GeoPoints.localCoord2D does not exist in workspace; interpreting Array{Main.GeoPoints.localCoord2D} as Array{Any}
└ @ JLD2 /Users/nobuaki/.julia/packages/JLD2/hbsZG/src/datasets.jl:128


This is the laziest ProduceOrLoad
Loading from ./tmp/seismicModelMarmousi.jld2


(ρ = [2.170699554722287 2.0245474895931075 … 1.0 1.0; 2.177730255448793 2.047502416431925 … 1.0 1.0; … ; 2.3907335223245925 2.3517540751240253 … 1.0 1.0; 2.3907335223245925 2.3253749707054134 … 1.0 1.0], Vpv = [4.101554566049527 3.7767721990957948 … 1.5 1.5; 4.117178345441762 3.8277831476265005 … 1.5 1.5; … ; 4.590518938499095 4.503897944720057 … 1.5 1.5; 4.590518938499095 4.445277712678697 … 1.5 1.5], Vph = [4.101554566049527 3.7767721990957948 … 1.5 1.5; 4.117178345441762 3.8277831476265005 … 1.5 1.5; … ; 4.590518938499095 4.503897944720057 … 1.5 1.5; 4.590518938499095 4.445277712678697 … 1.5 1.5], Vsv = [2.081243652839622 1.8214177592766359 … 0.0 0.0; 2.09374267635341 1.8622265181012005 … 0.0 0.0; … ; 2.4724151507992755 2.4031183557760456 … 0.0 0.0; 2.4724151507992755 2.3562221701429578 … 0.0 0.0], Vsh = [2.081243652839622 1.8214177592766359 … 0.0 0.0; 2.09374267635341 1.8622265181012005 … 0.0 0.0; … ; 2.4724151507992755 2.4031183557760456 … 0.0 0.0; 2.4724151507992755 2.35622217014